In [160]:
import numpy as np
from random import shuffle
from PIL.Image import open
from os import listdir
from typing import List

# Класс нейронки

In [161]:
class KN:
    learning_rate: float
    D: float = 1
    

    def __init__(self, input, clasters): 
        self.weights = np.random.uniform(low=-0.3, high=0.3, size=(clasters, input))

    def predict(self, vector: np.ndarray):
        dist: np.ndarray =  np.power((vector - self.weights), 2).sum(axis=1)
        winner_index = dist.argmin()
        return winner_index

    def train(self, vector: np.ndarray):
        winner_index = self.predict(vector)

        all_dists: np.ndarray = np.zeros(5, dtype=np.float32)
        rows, _ = self.weights.shape
        for index in range(0, rows):
            if index == winner_index:
                continue
            else:
                all_dists[index] = (np.power((vector - self.weights[index]), 2).sum())

        if self.D is None:
            max_dist_index = all_dists.argmax()
            self.D = all_dists[max_dist_index]
        
        all_errors = []
        for index in range(0, len(all_dists)):
            if index == winner_index or all_dists[index] < kn.D:
                delta: np.ndarray = self.learning_rate * (vector - self.weights[index])
                self.weights[index] += delta
                all_errors.append(np.abs(delta))
            
        all_errors = np.array(all_errors)
        return all_errors.sum()

# Загрузка датасета

In [162]:
def normalize(image: np.ndarray):
    new_image = []
    for rgb in image:
        rgb: np.ndarray
        if (rgb == [255,255,255]).all():
            new_image.append(0)
        else:
            new_image.append(1)

    return np.array(new_image)

In [163]:
dataset: List[tuple] = []
for file in listdir('data'):
    image = np.array(open(f'data/{file}'))
    x_max, y_max, _ = image.shape
    image = image.reshape((x_max*y_max, 3))
    image = normalize(image)
    dataset.append(tuple((file, image)))

In [164]:
test: List[tuple] = []
for file in listdir('test'):
    image = np.array(open(f'test/{file}'))
    x_max, y_max, _ = image.shape
    image = image.reshape((x_max*y_max, 3))
    image = normalize(image)
    test.append(tuple((file, image)))

# Обучение

In [165]:
kn = KN(2500, 5)
epoch = 200
kn.learning_rate = 0.8

all_deltas = []
epoch_count = 0
error_counter = np.zeros(shape=5)

for i in range(epoch):
    shuffle(dataset)
    delta: float = 0
    for _, image in dataset:
        delta += kn.train(image)
    
    delta = delta / len(dataset)
    all_deltas.append(round(delta, 5))
    if (delta < 0.05): break

    epoch_count += 1
    kn.learning_rate *= 0.9
    kn.D *= 0.9

# Кластеры

In [166]:
print('Обучающая выборка:')
all_class = { 0: {}, 1: {}, 2: {}, 3: {}, 4: {} }
for filename, image in dataset:
    classes = kn.predict(image)
    default_value = all_class[classes].get(filename.split(' ')[0], 0)
    new_value = default_value + 1
    all_class[classes][filename.split(' ')[0]] = new_value
    print(f'{filename}: Класс {classes}')

correct_classes = {}
for _class in range(0, len(all_class)):
    max_key = max(all_class[_class], key=all_class[_class].get)
    correct_classes[_class] = max_key
print(correct_classes)

print(f'\nИзменения на эпохе {all_deltas}')
print(f'Прошло эпох: {epoch_count}')

print('Тестовая выборка:')
error = 0
for filename, image in test:
    classes = kn.predict(image)
    if filename.split(' ')[0] != correct_classes[classes]: error += 1  
    print(f'{filename}: Класс {classes}')
    
        

print(f'Ошибка на тестовой выборке: {error / len(test)}')

Обучающая выборка:
Прямоугольник 8.png: Класс 0
круг 2.png: Класс 2
ромб 13.png: Класс 1
квадрат 14.png: Класс 4
круг 15.png: Класс 2
круг 4.png: Класс 2
квадрат 13.png: Класс 4
круг 16.png: Класс 2
Прямоугольник 12.png: Класс 0
треугольник 12.png: Класс 3
ромб 1.png: Класс 1
круг 17.png: Класс 2
треугольник 13.png: Класс 3
треугольник 7.png: Класс 3
Прямоугольник 17.png: Класс 0
круг 7.png: Класс 2
ромб 10.png: Класс 1
круг 8.png: Класс 2
треугольник 10.png: Класс 3
треугольник 18.png: Класс 3
квадрат 6.png: Класс 4
Прямоугольник 2.png: Класс 0
квадрат 11.png: Класс 4
ромб 18.png: Класс 1
треугольник 8.png: Класс 3
ромб 4.png: Класс 1
треугольник 5.png: Класс 3
Прямоугольник 4.png: Класс 0
треугольник 15.png: Класс 3
Прямоугольник 14.png: Класс 0
квадрат 1.png: Класс 4
круг 13.png: Класс 2
квадрат 10.png: Класс 4
Прямоугольник 5.png: Класс 0
квадрат 16.png: Класс 4
ромб 8.png: Класс 1
квадрат 4.png: Класс 4
ромб 17.png: Класс 1
круг 1.png: Класс 2
квадрат 8.png: Класс 4
квадрат 7.png: